In [1]:
"""Sample code for listing spack packages."""

'Sample code for listing spack packages.'

In [2]:
import importlib
import itertools
import re
import shutil
import sys
from dataclasses import dataclass
from pathlib import Path
from types import ModuleType

In [3]:
class Spack:
    """Spack interface class."""

    @dataclass
    class Modules:
        """Spack modules."""

        config: ModuleType
        repo: ModuleType

    def __init__(self) -> None:
        """Constructor."""
        self.modules = self.load_modules()
        self.repos = self.load_repo_list()
        self.packages = self.load_package_list()

    def load_modules(self) -> Modules:
        """Loads all required packages."""
        spack_root = Path(shutil.which("spack")).parent.parent
        lib_path = spack_root / "lib/spack"

        for path in [lib_path, lib_path / "external"]:
            if path not in sys.path:
                sys.path.append(str(path))

        return self.Modules(
            config=importlib.import_module('spack.config'),
            repo=importlib.import_module('spack.repo'),
        )

    def load_repo_list(self) -> list:
        """Load a list of all repos."""
        return list(
            map(self.modules.repo.Repo, self.modules.config.get("repos"))
        )

    class Package:
        """Wrapper for a spack package."""

        def __init__(self, modules, name) -> None:
            """Constructor."""
            self.modules = modules
            self.name = name

        def __str__(self) -> str:
            """String representation of the package."""
            return self.name

        def __repr__(self) -> str:
            """String representation of the package."""
            return self.name

        @property
        def dependencies(self):
            """Get a list of package dependencies."""
            raise NotImplementedError("this method is not implemented yet.")

        @property
        def latest(self):
            """Get the latest package version."""
            raise NotImplementedError("this method is not implemented yet.")

        @property
        def versions(self):
            """Get all package versions."""
            raise NotImplementedError("this method is not implemented yet.")

    def load_package_list(self) -> list[Package]:
        """Load a list of all packages."""
        return list(
            map(
                lambda package: self.Package(self.modules, package),
                itertools.chain.from_iterable(
                    list(
                        map(lambda repo: repo.all_package_names(), self.repos)
                    )
                ),
            )
        )

    def filter_packages(self, prefix) -> list[str]:
        """Filter packages based on a prefix."""
        regex = re.compile(fr"^{prefix}.*$")
        return list(filter(lambda p: regex.match(p.name), self.packages))

In [4]:
spack = Spack()

print(" Total packages:", len(spack.packages))

r_packages = spack.filter_packages("r-")
print("     R packages:", len(r_packages))

python_packages = spack.filter_packages("py-")
print("Python packages:", len(python_packages))

 Total packages: 7136
     R packages: 1118
Python packages: 2220
